In [2]:
import os, cv2, torch, shutil, csv
import pandas as pd
from IPython.display import display, Javascript
from google.colab import files

# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Ask user choice
choice = input("Do you want to detect objects in an 'image' or a 'video'? ").strip().lower()

if choice == 'image':
    from google.colab import files
    uploaded = files.upload()
    img_path = next(iter(uploaded))  # Get uploaded image path
    img = cv2.imread(img_path)
    results = model(img)
    detected_img = results.render()[0]
    detected_path = 'detected_image.jpg'
    cv2.imwrite(detected_path, detected_img)

    # Show result
    from IPython.display import Image as IPImage
    print("Detected Image:")
    display(IPImage(detected_path))
    files.download(detected_path)

    # Save results to CSV
    csv_file = 'image_detections.csv'
    csv_data = [['object_label', 'confidence']]
    for *_, conf, cls in results.pred[0]:
        csv_data.append([model.names[int(cls)], float(conf)])
    with open(csv_file, 'w', newline='') as f:
        csv.writer(f).writerows(csv_data)
    display(pd.read_csv(csv_file))
    files.download(csv_file)

elif choice == 'video':
    video_path = input("Enter the path to your video (e.g. /content/video.mp4): ").strip()
    frames_dir = 'frames'
    detected_dir = 'detected_frames'
    output_video = 'output_video_with_detections.mp4'
    csv_file = 'object_counts.csv'
    shutil.rmtree(frames_dir, ignore_errors=True)
    shutil.rmtree(detected_dir, ignore_errors=True)
    os.makedirs(frames_dir), os.makedirs(detected_dir)

    cap = cv2.VideoCapture(video_path)
    frame_interval = 5
    frame_rate = cap.get(cv2.CAP_PROP_FPS) / frame_interval
    frame_num, csv_data = 0, [['frame', 'object_label', 'confidence']]

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret: break
        if frame_num % frame_interval == 0:
            frame_name = f"frame_{frame_num:05d}.jpg"
            cv2.imwrite(os.path.join(frames_dir, frame_name), frame)
            results = model(frame)
            cv2.imwrite(os.path.join(detected_dir, frame_name), results.render()[0])
            for *_, conf, cls in results.pred[0]:
                csv_data.append([frame_name, model.names[int(cls)], float(conf)])
        frame_num += 1
    cap.release()

    # Save CSV
    with open(csv_file, 'w', newline='') as f:
        csv.writer(f).writerows(csv_data)

    # Install ffmpeg & create video
    !apt-get install -y -qq ffmpeg
    !ffmpeg -y -r {frame_rate} -pattern_type glob -i '{detected_dir}/frame_*.jpg' -c:v libx264 -pix_fmt yuv420p {output_video}

    # Display video
    display(Javascript(f"""
        var video = document.createElement('video');
        video.src = '{output_video}';
        video.controls = true;
        video.autoplay = true;
        video.loop = true;
        document.body.appendChild(video);
    """))

    # Show and download CSV + video
    display(pd.read_csv(csv_file))
    files.download(csv_file)
    files.download(output_video)

else:
    print("Invalid choice. Please type 'image' or 'video'.")


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-5-23 Python-3.11.12 torch-2.6.0+cu124 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Do you want to detect objects in an 'image' or a 'video'? video
Enter the path to your video (e.g. /content/video.mp4): /content/highway 1.mp4


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.am

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

<IPython.core.display.Javascript object>

,frame,object_label,confidence
0,frame_00000.jpg,truck,0.740240
1,frame_00000.jpg,truck,0.711961
2,frame_00000.jpg,car,0.505776
3,frame_00000.jpg,car,0.325868
4,frame_00000.jpg,truck,0.293862
...,...,...,...
324,frame_00295.jpg,bus,0.772856
325,frame_00295.jpg,truck,0.434252
326,frame_00300.jpg,bus,0.829980
327,frame_00300.jpg,bus,0.365191


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>